###  Libraries

In [1]:
import os
import sys
import torch
import numpy as np

from Datasets.Datasets_Functions import *
from Visualization import Visualization

from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

### Global Variables

In [2]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Visualization

In [3]:
visualization = Visualization()

### Load Dataset

In [4]:
dataset_name = "emnist_balanced_letters_A_J" # The available datasets are: emnist_balanced_digits, emnist_mnist, emnist_balanced_letters_A_J, emnist_balanced_letters_K_T, emnist_balanced_letters_U_Z, har, pendigits, waveform_v1, synthetic
module_name = "Datasets.Datasets"
batch_size = 256
n_clusters = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, module_name, batch_size, n_clusters)
print('Data_Shape is:', input_dim)
print('Batches Number is:', len(dataloader))
print(np.unique(labels))

Data_Shape is: 1
Batches Number is: 110
[10 11 12 13 14 15 16 17 18 19]


### Parameters

In [5]:
# Set latent dimension and negative slope
latent_dim = 10
negative_slope = 0

# Set pre-training epochs and learning rate
n_pret_epochs = 100
pret_lr = 1e-3

# Set alpha, momentum, gamma, training epochs and learning rate
alpha = 1.0
momentum = 0.9
gamma = 0.1
n_epochs = 100
lr = 5e-4

# Set use_pretrain and is_mlp_ae
use_pretrain = True
is_mlp_ae = False

### Create Autoencoder Model

In [6]:
if(is_mlp_ae):
    idec_autoencoder = IDECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim , negative_slope=negative_slope)
else:  
    input_dim = 1
    idec_autoencoder = IDECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope) 
idec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
idec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
idec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha, gamma=gamma)
idec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
idec_autoencoder.set_path()
idec_autoencoder = idec_autoencoder.to(device)

### Pretrain Autoencoder

In [7]:
if(use_pretrain):
    idec_autoencoder.pretrain_autoencoder()
    idec_autoencoder.save_pretrained_weights()
else:
    model_save_path = idec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    idec_autoencoder.load_state_dict(torch.load(model_save_path))

Epoch: 0, Loss: 14.654978
Epoch: 1, Loss: 6.306714
Epoch: 2, Loss: 4.648780
Epoch: 3, Loss: 3.902864
Epoch: 4, Loss: 3.516965
Epoch: 5, Loss: 3.273267
Epoch: 6, Loss: 3.101950
Epoch: 7, Loss: 2.953062
Epoch: 8, Loss: 2.832125
Epoch: 9, Loss: 2.744569
Epoch: 10, Loss: 2.673469
Epoch: 11, Loss: 2.582670
Epoch: 12, Loss: 2.520216
Epoch: 13, Loss: 2.477683
Epoch: 14, Loss: 2.418638
Epoch: 15, Loss: 2.376636
Epoch: 16, Loss: 2.363553
Epoch: 17, Loss: 2.313520
Epoch: 18, Loss: 2.299832
Epoch: 19, Loss: 2.247874
Epoch: 20, Loss: 2.220919
Epoch: 21, Loss: 2.186193
Epoch: 22, Loss: 2.167298
Epoch: 23, Loss: 2.151444
Epoch: 24, Loss: 2.105300
Epoch: 25, Loss: 2.100503
Epoch: 26, Loss: 2.071942
Epoch: 27, Loss: 2.056818
Epoch: 28, Loss: 2.040929
Epoch: 29, Loss: 2.025623
Epoch: 30, Loss: 2.026499
Epoch: 31, Loss: 2.013193
Epoch: 32, Loss: 1.984899
Epoch: 33, Loss: 1.965605
Epoch: 34, Loss: 1.972945
Epoch: 35, Loss: 1.943379
Epoch: 36, Loss: 1.926334
Epoch: 37, Loss: 1.914158
Epoch: 38, Loss: 1.91

### Initialize the clustering layer using k-means

In [ ]:
idec_autoencoder.kmeans_initialization()

### Train Autoencoder

In [ ]:
idec_autoencoder.train_autoencoder()

### Save Clustering Results

In [ ]:
General_Functions().save_excel(idec_autoencoder.data_dir_path, idec_autoencoder.df_eval)

### Plot Clustering Results

In [ ]:
cluster_centers = idec_autoencoder.get_cluster_centers()
_, idec_autoencoder_reduced_data, labels = idec_autoencoder.get_latent_data()
visualization.plot_tsne(idec_autoencoder_reduced_data, labels, labels, cluster_centers, idec_autoencoder.data_dir_path)